In [30]:
import os
from dotenv import load_dotenv
load_dotenv()

import json

from get_articles import get_data
from get_articles import process_articles

from importlib import reload
reload(get_data)
reload(process_articles)

import pprint
pp = pprint.PrettyPrinter(indent=2)

In [3]:
home = os.getenv("PROJ_HOME")

institutional_data_path = os.path.join(home, "data/institutional")
metadata_path = os.path.join(home, "data/subject_matter/metadata")
article_ids_path = os.path.join(home, "data/cache/article_ids.json")
data_out_dir = os.path.join(home, "data/subject_matter/encyclopedia")

In [368]:
# get encyclopedia metadata

def get_metadata(sources=["advanced"], path=None):
    articles = {}

    for source in sources:
        articles_in_source = get_data.get_encyclopedia_metadata(source, path)
        articles[source] = articles_in_source

    # remove articles with empty title fields
    for source in sources:
        for article in articles[source]:
            if article['title'] == '':
                articles[source].remove(article)

    if not os.path.exists(path):
        os.makedirs(path)

    for source in sources:
        filepath = os.path.join(path, source + '.json')

        if os.path.exists(filepath):
            os.remove(filepath)

        with open(filepath, "w") as outfile:
            outfile.write(json.dumps(articles[source], indent=2))

    return articles

In [369]:
advanced_metadata = get_metadata(path=metadata_path)

In [4]:
# load article_ids to get

with open(article_ids_path, "r") as infile:
    article_ids = json.load(infile)

In [ ]:
# get the articles from the API
articles = get_data.get_articles(article_ids)

In [ ]:
# clean articles and save to a dataframe

df = process_articles.clean_articles(articles)

In [55]:
# save dataframe to disk as csv

filepath = os.path.join(home, "data/subject_matter/encyclopedia/data.csv")
df.to_csv(filepath)